In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


### Import packages.

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from statsmodels.tsa.stattools import acf
from statsmodels.tsa.arima_model import ARIMA

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


### Import the data

In [ ]:
mobility = pd.read_csv('/content/drive/My Drive/or_data/population.csv',index_col=0)

target_30_cities = pd.read_csv('/content/drive/My Drive/or_data/sample_submittion_round1 (1).csv',index_col=0)

In [ ]:
target_30_cities_list = list(target_30_cities.columns)

In [ ]:
target_30_cities.columns

Index(['福岡県福岡市中央区', '東京都千代田区', '島根県松江市', '埼玉県秩父郡長瀞町', '高知県吾川郡仁淀川町', '群馬県安中市',
       '青森県平川市', '宮城県栗原市', '北海道砂川市', '山梨県南都留郡道志村', '福島県南会津郡檜枝岐村',
       '長野県北佐久郡軽井沢町', '大阪府大阪市北区', '愛知県常滑市', '広島県三原市', '岐阜県大野郡白川村', '群馬県吾妻郡草津町',
       '北海道空知郡南富良野町', '千葉県千葉市美浜区', '千葉県浦安市', '新潟県南魚沼郡湯沢町', '茨城県鹿嶋市', '奈良県天理市',
       '秋田県仙北郡美郷町', '福井県勝山市', '佐賀県鳥栖市', '沖縄県那覇市', '滋賀県犬上郡甲良町', '東京都西多摩郡奥多摩町',
       '長野県東筑摩郡筑北村'],
      dtype='object')

In [ ]:
target_mobility = mobility[['福岡県福岡市中央区', '東京都千代田区', '島根県松江市', '埼玉県秩父郡長瀞町', '高知県吾川郡仁淀川町', '群馬県安中市',
       '青森県平川市', '宮城県栗原市', '北海道砂川市', '山梨県南都留郡道志村', '福島県南会津郡檜枝岐村',
       '長野県北佐久郡軽井沢町', '大阪府大阪市北区', '愛知県常滑市', '広島県三原市', '岐阜県大野郡白川村', '群馬県吾妻郡草津町',
       '北海道空知郡南富良野町', '千葉県千葉市美浜区', '千葉県浦安市', '新潟県南魚沼郡湯沢町', '茨城県鹿嶋市', '奈良県天理市',
       '秋田県仙北郡美郷町', '福井県勝山市', '佐賀県鳥栖市', '沖縄県那覇市', '滋賀県犬上郡甲良町', '東京都西多摩郡奥多摩町',
       '長野県東筑摩郡筑北村']]

In [ ]:
target_mobility

,福岡県福岡市中央区,東京都千代田区,島根県松江市,埼玉県秩父郡長瀞町,高知県吾川郡仁淀川町,群馬県安中市,青森県平川市,宮城県栗原市,北海道砂川市,山梨県南都留郡道志村,福島県南会津郡檜枝岐村,長野県北佐久郡軽井沢町,大阪府大阪市北区,愛知県常滑市,広島県三原市,岐阜県大野郡白川村,群馬県吾妻郡草津町,北海道空知郡南富良野町,千葉県千葉市美浜区,千葉県浦安市,新潟県南魚沼郡湯沢町,茨城県鹿嶋市,奈良県天理市,秋田県仙北郡美郷町,福井県勝山市,佐賀県鳥栖市,沖縄県那覇市,滋賀県犬上郡甲良町,東京都西多摩郡奥多摩町,長野県東筑摩郡筑北村
2019/01/01,243200.0,293700.0,210500.0,10100.0,5720.0,70700.0,38300.0,72300.0,22800.0,1770.0,590.0,28300.0,244000.0,71100.0,101500.0,2350.0,8500.0,2750.0,199800.0,206700.0,14800.0,79200.0,80900.0,24900.0,24000.0,95900.0,368400.0,11100.0,6300.0,5600.0
2019/01/02,267200.0,383700.0,211200.0,10700.0,5590.0,72000.0,39300.0,73400.0,24400.0,1970.0,660.0,28200.0,301000.0,74700.0,103800.0,2530.0,8600.0,2820.0,211600.0,210700.0,16200.0,78800.0,83500.0,24100.0,24900.0,103600.0,371300.0,12000.0,6400.0,5900.0
2019/01/03,269400.0,377700.0,212300.0,10600.0,5480.0,70800.0,38500.0,72300.0,24400.0,1990.0,540.0,27200.0,297000.0,72600.0,102600.0,2500.0,8400.0,2910.0,207300.0,208400.0,16600.0,77300.0,82300.0,24300.0,24700.0,104300.0,369100.0,10600.0,6300.0,5700.0
2019/01/04,304000.0,659700.0,214300.0,9900.0,5630.0,70400.0,38000.0,72500.0,24300.0,2000.0,500.0,26000.0,391000.0,73700.0,102400.0,2490.0,8500.0,2800.0,223200.0,216600.0,15500.0,78400.0,83800.0,23900.0,25000.0,103400.0,380400.0,11700.0,6200.0,5800.0
2019/01/05,288000.0,427700.0,211800.0,10100.0,5720.0,70800.0,39200.0,71900.0,24100.0,1920.0,480.0,25200.0,336000.0,72300.0,100400.0,2190.0,8000.0,2860.0,213100.0,212000.0,14600.0,77600.0,84300.0,23200.0,24500.0,101000.0,373600.0,10800.0,6300.0,5500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020/08/21,315000.0,761400.0,215400.0,10000.0,5830.0,70000.0,39000.0,74000.0,23300.0,2200.0,680.0,27500.0,438000.0,72100.0,103300.0,1850.0,7300.0,2990.0,230300.0,223500.0,10200.0,78500.0,82800.0,23900.0,25400.0,101600.0,375500.0,11600.0,6800.0,5170.0
2020/08/22,273700.0,343400.0,211200.0,10100.0,5630.0,68400.0,38300.0,72600.0,23300.0,2700.0,750.0,27700.0,300000.0,70000.0,100900.0,1900.0,7500.0,3000.0,208900.0,210400.0,10100.0,75700.0,78800.0,23900.0,25000.0,97000.0,366300.0,10600.0,7100.0,5090.0
2020/08/23,256500.0,260400.0,208200.0,9600.0,5690.0,66600.0,37400.0,71300.0,23000.0,2250.0,690.0,26800.0,261000.0,69600.0,99500.0,1830.0,7300.0,2960.0,199100.0,201100.0,9700.0,74300.0,76000.0,23400.0,24800.0,92700.0,356900.0,9900.0,6300.0,5050.0
2020/08/24,314000.0,783500.0,215300.0,9700.0,5900.0,69100.0,38500.0,73800.0,22900.0,1990.0,700.0,26400.0,437000.0,71500.0,103200.0,1780.0,7100.0,2860.0,228100.0,221100.0,10100.0,78500.0,82300.0,23800.0,25200.0,101200.0,371300.0,11400.0,6400.0,5180.0


### Build an ARIMA(1, 1, 0) model on training dataset and forecast on the testing dataset

### Define a function to get the RMSE of the last 120 days' prediction and its population mobility prediction for the first half of September, 2020. 

In [ ]:
def RMSE_prediction_city(city_name, dataset):
    
    from numpy.random import seed
    seed(123)
    # Obtain the population mobility of the specified city.
    
    training_data = target_mobility[city_name]

    
    # Create training and testing dataset by splitting the time series into 2 contiguous parts(training,testing)
    # Keep the population mobility of the last 120 days until 2020-08-25 for testing and the rest for training.
    
    train = training_data[:483]
    test = training_data[483:]

    # Build an ARIMA(1, 1, 0) model on training dataset and forecast on the testing dataset.
    model = ARIMA(train, order=(1, 1, 0))  
    fitted = model.fit(disp=-1)  
    fc, se, conf = fitted.forecast(120, alpha=0.05)  # 95% conf
    forecast = fc
    actual = test.values

    rmse = np.mean((forecast - actual)**2)**.5 
    # from sklearn.metrics import mean_squared_error 
    # rmse_2 = mean_squared_error(actual, forecast)**0.5

    fc_new, se_new, conf_new = fitted.forecast(120+21, alpha=0.05)  # 95% conf
    new_days_predict = fc_new
    final_predcition = new_days_predict[126:]

    
    return ({ 
            'rmse_ARIMA110':rmse,
            'prediction_ARIMA110': final_predcition
           })
    

In [ ]:
f = RMSE_prediction_city('福岡県福岡市中央区',target_mobility)
f

/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)


{'prediction_ARIMA110': array([291689.00077769, 291777.79745586, 291866.59413403, 291955.3908122 ,
        292044.18749037, 292132.98416854, 292221.78084671, 292310.57752488,
        292399.37420305, 292488.17088122, 292576.96755939, 292665.76423756,
        292754.56091573, 292843.3575939 , 292932.15427207]),
 'rmse_ARIMA110': 30528.624306048965}

- Get a list of dictionaries and each dictionary contains RMSE for the prediction of corresponding city and its prediction on the population mobility of first half of September. The list contains all 30 cities.

In [ ]:
import time
start_time = time.time()
RMSE_prediction_30_cities = [RMSE_prediction_city(x,target_mobility) for x in target_30_cities_list]

RMSE_prediction_30_cities
print("--- %s seconds ---" % (time.time() - start_time))

/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.6/dist-packa

--- 1.4649624824523926 seconds ---


/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)


### Transform the list of dictionaries to a dataframe and add a new column of city_name as the first column of the dataframe. 

In [ ]:
df_rmse_prediction_ARIMA110 = pd.DataFrame(RMSE_prediction_30_cities)




idx = 0
new_col = target_30_cities_list  
df_rmse_prediction_ARIMA110.insert(loc=idx, column='city name', value=new_col)
df_rmse_prediction_ARIMA110

,city name,rmse_ARIMA110,prediction_ARIMA110
0,福岡県福岡市中央区,30528.624306,"[291689.0007776923, 291777.7974558623, 291866...."
1,東京都千代田区,240953.918975,"[541474.5447972174, 541899.0261250181, 542323...."
2,島根県松江市,3418.249308,"[214329.27284673945, 214334.8808322517, 214340..."
3,埼玉県秩父郡長瀞町,731.055640,"[9051.490796733924, 9049.63982111855, 9047.788..."
4,高知県吾川郡仁淀川町,154.607569,"[5901.985840501036, 5902.335277020811, 5902.68..."
5,群馬県安中市,1786.851694,"[67977.65419684578, 67972.8658390766, 67968.07..."
6,青森県平川市,778.848922,"[38509.530141077754, 38509.57482292609, 38509...."
7,宮城県栗原市,1073.643159,"[74464.5603750912, 74467.95965015446, 74471.35..."
8,北海道砂川市,1027.002219,"[22650.289163581496, 22650.005803252698, 22649..."
9,山梨県南都留郡道志村,494.078076,"[1894.6388591515613, 1894.8319884047003, 1895...."


In [ ]:
len(df_rmse_prediction_ARIMA110['prediction_ARIMA110'][0])

15

In [ ]:
df_rmse_prediction_ARIMA110['prediction_ARIMA110'][0]

array([291689.00077769, 291777.79745586, 291866.59413403, 291955.3908122 ,
       292044.18749037, 292132.98416854, 292221.78084671, 292310.57752488,
       292399.37420305, 292488.17088122, 292576.96755939, 292665.76423756,
       292754.56091573, 292843.3575939 , 292932.15427207])

### Save the results.

In [ ]:
df_rmse_prediction_ARIMA110.to_csv('/content/drive/My Drive/or_data/df_rmse_prediction_ARIMA110.csv',index=False,encoding='utf_8_sig')